In [18]:
import numpy as np

import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import umap
from torch.nn.functional import mse_loss
from tqdm import tqdm
from sklearn.decomposition import PCA

from autoencoder import Autoencoder
from solver import *

In [19]:
autoencoder_types = [
    ('l_models', 'SAE'),
    ('l_models', 'DAE'),
    ('nl_models', 'SAE'),
    ('nl_models', 'DAE')
]

In [20]:
average_last_losses = {}

for ae_type in autoencoder_types:
    last_losses = []
    
    for model_id in range(10):
        file_path = f'/home/david/sparsify_models/{ae_type[0]}/{model_id}/{ae_type[1]}/all_train_losses.npy'
        losses = np.load(file_path)
        # Average loss in the last training epoch
        last_loss = np.mean(losses[-1])
        last_losses.append(last_loss)
    
    average_last_losses[ae_type] = np.mean(last_losses)

for ae_type, avg_loss in average_last_losses.items():
    print(f"{ae_type}: Average Training Loss = {avg_loss:.5f}")

('l_models', 'SAE'): Average Training Loss = 0.01733
('l_models', 'DAE'): Average Training Loss = 0.01925
('nl_models', 'SAE'): Average Training Loss = 0.01735
('nl_models', 'DAE'): Average Training Loss = 0.01926


In [21]:
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True, num_workers=6)

mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(mnist_test, batch_size=128, shuffle=False, num_workers=6)

In [22]:
def load_model(model_path, model_type, epoch):
    n_input = 28*28
    n_layers = 3
    sae_n_hidden_ls = [512, 128, 32]

    size_ls = [4, 4, 4, 4, 4, 10,
            10, 10, 10, 10, 16, 16,
            16, 16, 16, 16, 16, 24,
            24, 24, 24, 24, 24, 24, 
            32, 32, 32, 32, 32, 32,
            32, 32, 32, 32, 32, 32, 
            32, 32, 32, 32, 32, 32, 
            32, 32, 32, 32, 32, 32, 
            32, 32]
    
    dae_n_hidden_ls = [512, 128, size_ls[epoch]]
    
    if model_type == 'SAE':
        model = Autoencoder(n_input, sae_n_hidden_ls, n_layers)
    else:
        model = Autoencoder(n_input, dae_n_hidden_ls, n_layers)
    weights = torch.load(f"{model_path}/{model_type}/model_weights_epoch{epoch}.pth")
    model.load_state_dict(weights)
    return model

In [27]:
average_last_losses = {}
for ae_type in autoencoder_types:
    last_losses = []
    
    for model_id in range(10):
        file_path = f'/home/david/sparsify_models/{ae_type[0]}/{model_id}/'
        model = load_model(file_path, ae_type[1], 49)
        # Average loss in the last training epoch
        test_loss, _, _ = test(model, test_loader, device='cpu')
        last_losses.append(test_loss)
    
    average_last_losses[ae_type] = np.mean(last_losses)

for ae_type, avg_loss in average_last_losses.items():
    print(f"{ae_type}: Average Test Loss = {avg_loss:.5f}")

/tmp/ipykernel_8623/2098548407.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(f"{model_path}/{model_type}/model_weights_epoch{epoch}.pth")


====> Test set loss: 0.0997
====> Test set loss: 0.1010
====> Test set loss: 0.0974
====> Test set loss: 0.1039
====> Test set loss: 0.1002
====> Test set loss: 0.1010
====> Test set loss: 0.1014
====> Test set loss: 0.0988
====> Test set loss: 0.1002
====> Test set loss: 0.0994
====> Test set loss: 0.0990
====> Test set loss: 0.1045
====> Test set loss: 0.1039
====> Test set loss: 0.0997
====> Test set loss: 0.0989
====> Test set loss: 0.0986
====> Test set loss: 0.1025
====> Test set loss: 0.1003
====> Test set loss: 0.1024
====> Test set loss: 0.1035
====> Test set loss: 0.1030
====> Test set loss: 0.1007
====> Test set loss: 0.1009
====> Test set loss: 0.0989
====> Test set loss: 0.1004
====> Test set loss: 0.0984
====> Test set loss: 0.1015
====> Test set loss: 0.1011
====> Test set loss: 0.0999
====> Test set loss: 0.0966
====> Test set loss: 0.1019
====> Test set loss: 0.1139
====> Test set loss: 0.0998
====> Test set loss: 0.1000
====> Test set loss: 0.1022
====> Test set loss: